In [3]:
import functools as ft
import numpy as np
import pandas as pd
import multiprocessing as mp
import sys
import re
import csv
import collections as co
import itertools as it
import umap
import hdbscan
import time 
#import progressbar as pb
import scipy.spatial.distance as ssd

In [4]:
class wrapper(object):
    
    pass

In [63]:
class vectorizer(object):
    
    #def __init__(self, procs = 8, k = 7, convert = 0, row = 0, index = [], exist = co.defaultdict(int)):
    def __init__(self, k = 7, convert = 0):
    
        self.k = k
        self.convert = convert
        self.index = [] 
        #self.index = index
        self.exist = co.defaultdict(int) 
        #self.exist = exist
        self.keys = list(self.exist.keys())
        self.col = len(self.keys)
        self.row = 0
        #self.row = row
        self.matrix = np.empty((self.row, self.col, ),dtype = "float32")
        self.amino = co.defaultdict(str, {
            'AAA':'K', 'AAC':'N', 'AAG':'K', 'AAT':'N',
            'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
            'AGA':'R', 'AGC':'S', 'AGG':'R', 'AGT':'S',
            'ATA':'I', 'ATC':'I', 'ATG':'M', 'ATT':'I',
            'CAA':'Q', 'CAC':'H', 'CAG':'Q', 'CAT':'H',
            'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
            'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
            'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
            'GAA':'E', 'GAC':'D', 'GAG':'E', 'GAT':'D',
            'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
            'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
            'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',    
            'TAA':'Y', 'TAC':'*', 'TAG':'*', 'TAT':'Y',
            'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
            'TGA':'*', 'TGC':'C', 'TGG':'W', 'TGT':'C',
            'TTA':'L', 'TTC':'F', 'TTG':'L', 'TTT':'F'
        })
                
    def translate(self, read):
    
        chain = ''

        for i in range(len(read) - 2):
            trip = read[i:i+3]
            chain += self.amino[trip]

        return(chain)
    
    
    def adjust_to_data(self, infile):
    
        #widgets = ['Data adjustment:    ', pb.AnimatedMarker()] 
        #bar = pb.ProgressBar(widgets=widgets).start() 
        data = pd.read_csv(infile, chunksize = 10000, sep = ';', na_filter = False, header = None)
        self.row = 0
        #t = 0
        
        for chunk in data:

            for info, read in chunk.itertuples(index=False, name=None):
                
                #name = info
                name = (info.split('|')[0][1:], info.split('|')[5])

                self.index.append(name)
                del name
                
                if self.convert == 1:
                    #seq = self.translate(re.sub('[^ACGT]+', '', read))
                    seq = self.translate(read)
                    del read
                    
                    num = len(seq) - self.k + 1
                    
                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        self.exist[kmer] = 0
                    
                else:
                    #seq = re.sub('[^ACGT]+', '', read)
                    seq = read
                    del read

                    num = len(seq) - self.k + 1

                    if re.match('^[ACGT]*$', seq): 
                        for i in range(num):
                            kmer = seq[i:i+self.k]
                            self.exist[kmer] = 0
                    else:
                        for i in range(num):
                            kmer = seq[i:i+self.k]
                            if re.match('^[ACGT]*$', kmer): 
                                self.exist[kmer] = 0

                #t = t + 1
                self.row = self.row + 1
                #bar.update(t)
            
        self.keys = list(self.exist.keys())
        self.col = len(self.keys)
        self.matrix = np.empty((self.row, self.col, ), dtype="float32")
        
        #bar.finish()
        del seq
    
    def calculate_frequence(self, infile):
        
        #widgets = [' [', pb.Timer(format= 'Vector calculation: %(elapsed)s'), '] ', pb.Bar('*'),' (', pb.ETA(), ') ', ] 
        #widgets = [pb.Timer(format= 'Vector calculation: '), pb.Bar('#'),' (', pb.ETA(), ')'] 
        #bar = pb.ProgressBar(max_value=self.row, widgets=widgets).start() 
        n = 0
        #t = 0
        data = pd.read_csv(infile, chunksize = 10000, sep = ';', na_filter = False, header = None)
        
        for chunk in data:

            for info, read in chunk.itertuples(index=False, name=None):

                if self.convert == 1:
                    #seq = self.translate(re.sub('[^ACGT]+', '', read))
                    seq = self.translate(read)
                    del read
                
                    counts = self.exist.copy()
                    num = len(seq) - self.k + 1

                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        counts[kmer] += 1
                            
                else:
                    #seq = re.sub('[^ACGT]+', '', read)
                    seq = read
                    del read
                
                    counts = self.exist.copy()
                    num = len(seq) - self.k + 1

                    if re.match('^[ACGT]*$', seq): 
                        for i in range(num):
                            kmer = seq[i:i+self.k]
                            counts[kmer] += 1
                    else:
                        for i in range(num):
                            kmer = seq[i:i+self.k]
                            if re.match('^[ACGT]*$', kmer): 
                                counts[kmer] += 1

                vector = np.array(list(counts.values()), dtype = "float32")/num
                self.matrix[n] = vector
                
                n = n + 1
                #t = t + 1
                #bar.update(t)
                
                counts.clear()
                del vector
                del seq
                del counts

        #bar.finish()
            
    def get_index(self):
        
        return(self.index)
    
    
    def get_keys(self):
        
        return(self.keys)
    
    
    def get_matrix(self):
        
        return(self.matrix)

In [64]:
class extractor(object):
    
    pass

In [67]:
def main():

    print("Welcome. Read input and settings file.", end = ' ')
    #error checkup?
    
    infile = '../../../Desktop/Masterthesis_V5/A_HA.csv'   
    outfile = 'output.csv'
    setfile = 'settings.csv'

    settings = pd.read_csv(setfile, sep = ',', na_filter = False, index_col = 0)
    parameter = settings.loc[4].to_list()
    
    print("Finished.")
    
    #infile = sys.argv[1]
    #outfile = sys.argv[2]

    start = time.perf_counter()

    print("Nucleotide k-mer frequency calculation.", end = ' ')

    freq_nt = vectorizer(k = parameter[0].item(), convert = 0)
    freq_nt.adjust_to_data(infile)
    freq_nt.calculate_frequence(infile)

    matrix_nt = freq_nt.get_matrix()
    index_nt = freq_nt.get_index()   
    keys_nt = freq_nt.get_keys()

    del freq_nt

    print("Finished.")

    print("Running UMAP for dimension reduction.", end = ' ')

    matrix_nt_red = umap.UMAP(
        n_neighbors = parameter[1].item(),
        min_dist = parameter[2].item(),
        n_components = parameter[3].item(),
        random_state = parameter[4].item(),
        metric = parameter[5],
    ).fit_transform(matrix_nt)

    del matrix_nt

    print("Finished.")

    print("Aminoacid k-mer frequency calculation.", end = ' ')

    freq_aa = vectorizer(k = parameter[6].item(), convert = 1)
    freq_aa.adjust_to_data(infile)
    freq_aa.calculate_frequence(infile)

    matrix_aa = freq_aa.get_matrix()
    index_aa = freq_aa.get_index()
    keys_aa = freq_aa.get_keys()

    del freq_aa

    print("Finished.")

    print("Running UMAP for dimension reduction.", end = ' ')

    matrix_aa_red = umap.UMAP(
        n_neighbors = parameter[7].item(),
        min_dist = parameter[8].item(),
        n_components = parameter[9].item(),
        random_state = parameter[10].item(),
        metric = parameter[11],
    ).fit_transform(matrix_aa)

    del matrix_aa

    print("Finished.")

    matrix_aa_ind = pd.DataFrame(matrix_aa_red, index = index_aa)
    matrix_nt_ind = pd.DataFrame(matrix_nt_red, index = index_nt)

    matrix = pd.concat([matrix_nt_ind, matrix_aa_ind], axis=1, copy = False, ignore_index = True) #falsches Ergebnis? checken ob ignore_index = Fehler

    print("Running HDBscan for clustering.", end = ' ')

    #end whitespace als bessere alternative
    #link in teams

    matrix_clust = hdbscan.HDBSCAN(
        min_samples = parameter[12].item(), #larger the value the more conservative the clustering (more points will be declared as noise)
        min_cluster_size = parameter[13].item(), #minimum size that can become a cluster
        cluster_selection_epsilon = parameter[14].item(), #don't seperate clusters with a distance less than value
        alpha = parameter[15].item(), #don't mess with this
    ).fit(matrix)

    print("Finished.")

    print("Centroid extraction.", end = ' ')

    clusterlabel = matrix_clust.labels_

    index = pd.MultiIndex.from_tuples(index_nt, names=["accession", "subtype"])

    clusters = pd.DataFrame(zip(clusterlabel, ['false'] * len(clusterlabel)), index = index, columns = ['cluster', 'centroid'])

    num = clusters['cluster'].max()+1
    values = ['true']*num
    accessions = []

    #mixed = []
    #res = [i for i in x if i != 'X']
    #'X aus der liste löschen und nochmal schauen um zu sehen, ob ansonsten alle gleich'
    #2. print ausgabe sowas wie: ignoring mixed and nulls result in: 
    
    sanity = []
    for i in range(num):

        query = clusters[clusters.cluster == i]
        match = query.index.values.tolist()
        sub = matrix.filter(items = match, axis=0)
        dist = ssd.cdist(sub, sub, metric = parameter[16])
        accessions.append(pd.DataFrame(dist, columns = match, index = match, dtype = 'float32').mean().idxmin())

        subs = {'H': [], 'N': []} 
        for x, y in match: 
            if re.match('^[H][0-9]+N[0-9]+$', y): 
                subs['H'].append(re.search('[H][0-9]+', y).group(0))
                subs['N'].append(re.search('[N][0-9]+', y).group(0))
            else: #mixed types sind nicht zwingend falsch, schwierig zu bewerten ob sie beachtet werden oder nicht
                subs['H'].append('X')
                subs['N'].append('X')

        if len(set(subs['H'])) == 1 and len(set(subs['N'])) == 1:
            sanity.append(2)
        elif len(set(subs['H'])) == 1:
            sanity.append(1)
        elif len(set(subs['N'])) == 1:
            sanity.append(0)

    centroids = pd.DataFrame(values, columns=['centroid'], index = accessions)

    #result = pd.concat([clusters, centroids], axis=1, copy = False).fillna('false')
    clusters.update(centroids)

    clusters.reset_index(level=['subtype']).sort_values(by=['cluster', 'subtype']).to_csv(outfile, index=True, header=True, sep=',')

    stop = time.perf_counter()

    print("Finished.")
    print(f"Clustering done in {stop - start:0.4f} seconds.")
    diagnostic = co.Counter(clusterlabel)
    print(f"{str(len(clusterlabel))} sequences, {str(diagnostic[-1])} unclustered, {str(len(set(diagnostic)))} cluster.")

    N = sanity.count(0) + sanity.count(2)
    H = sanity.count(1) + sanity.count(2)
    
    print(f"{N} clusters containing matching NA types.")
    print(f"{H} clusters containing matching HA types.")

In [68]:
if __name__ == "__main__":

    main()

Welcome. Read input and settings file. Finished.
Nucleotide k-mer frequency calculation. Finished.
Running UMAP for dimension reduction. Finished.
Aminoacid k-mer frequency calculation. Finished.
Running UMAP for dimension reduction. Finished.
Running HDBscan for clustering. Finished.
Centroid extraction. Finished.
Clustering done in 1093.8409 seconds.
56600 sequences, 31 unclustered, 826 cluster.
555 clusters containing matching NA types.
617 clusters containing matching HA types.
